In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(
    '/kaggle/input/movies/movies_metadata.csv',
    engine='python',
    sep=',',
    escapechar='\\',
    on_bad_lines='skip'
)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df = df[["title", "genres", "tagline", "vote_average","popularity","overview"]]

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(subset=['title'])

In [ ]:
df['overview'] = df['overview'].fillna('')

In [ ]:
df.iloc[0]['genres']

In [ ]:
import ast

In [ ]:
df['genres'] = df['genres'].apply(lambda x: " ".join([i['name'] for i in ast.literal_eval(x)]))

In [ ]:
df['tagline'] = df['tagline'].fillna('')

In [ ]:
df.isnull().sum()

In [ ]:
df['tags'] = df['overview'] + ' ' + df['genres'] + ' ' + df['tagline']

In [ ]:
df.head()

In [ ]:
df['tags'][1]

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
  text = str(text).lower()
  text = re.sub(r'[^a-zA-Z\s]','',text)
  words = text.split()
  words = [word for word in words if word not in stop_words]
  words = [lemmatizer.lemmatize(word) for word in words]
  return " ".join(words)

In [ ]:
df['tags'] = df['tags'].apply(preprocess_text)

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
indices = pd.Series(df.index,index = df['title']).drop_duplicates()
indices

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(max_features=50000,ngram_range=(1,2),stop_words='english')

In [ ]:
tfidf_matrix = tfidf.fit_transform(df['tags'])

In [ ]:
tfidf_matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def recommend(title, n = 10):
  if title not in indices:
    return ['Movie not found']

  idx = indices[title]
  sim_score = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
  similar_idx = sim_score.argsort()[::-1][1:n+1]
  return df['title'].iloc[similar_idx]


In [ ]:
recommend('Avatar 2',5)

In [ ]:
import pickle

pickle.dump(tfidf_matrix,open('tfidf_matrix.pkl','wb'))

pickle.dump(indices,open('indices.pkl','wb'))

df.to_pickle('df.pkl')

pickle.dump(tfidf,open('tfidf.pkl','wb'))